In [1]:
import sys; sys.path.append("..")
import os
import glob
import ml_logger
from datetime import datetime; now = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
from collections import defaultdict
import ujson
import numpy as np
import tensorflow as tf 
from smoothdoq import denoiser, noise, binned
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt; plt.style.use("dark_background")
from ml_logger import logbook as ml_logbook

In [2]:
logdir = "../tflogs/" + now
logbook_config = ml_logbook.make_config(
    logger_dir=logdir,
#     write_to_console=
    tensorboard_config=dict(logdir=logdir)
)
logger = ml_logbook.LogBook(logbook_config)

In [3]:
training_data = []
for fn in glob.glob("../data/simulated/normal/*.json"):
    with open(fn, "r") as io:
        batch = ujson.load(io)
        for record in batch:
            training_data.append(record)

In [4]:
def preprocess_batch(batch: list, logits=True, eps=1e-12):
    Dx = [b["sample"] for b in batch]
    mask = denoiser.compute_mask(Dx)
    Dy = [b["pdf"] for b in batch]
    max_logits = - tf.math.log(eps)

    # preprocess input sequences of probabilities to logits
    x = tf.constant(
        pad_sequences(Dx, dtype="float", padding="post"), tf.float32
    )
    logits_x = tf.math.log(eps + x)
    logits_x -= tf.reduce_mean(logits_x, -1, keepdims=True)
    logits_x = tf.clip_by_value(logits_x, -max_logits, max_logits)
    
    # preprocess logits of true pdf
    y = tf.constant(
        pad_sequences(Dy, dtype="float", padding="post"), tf.float32
    )
    logits_y = tf.math.log(eps + y)
    logits_y -= tf.reduce_mean(logits_y, -1, keepdims=True)
    logits_y = tf.clip_by_value(logits_y, -max_logits, max_logits)

    return logits_x, x, logits_y, y, mask

In [5]:
batch_size = 32
epochs = 100

def batches():
    N  = len(training_data) // batch_size
    for i in range(N):
        idx = range(i * batch_size, (i + 1) * batch_size)
        batch = [training_data[k] for k in idx]
        yield preprocess_batch(batch)

dataset = tf.data.Dataset.from_generator(
    batches,
    args=[],
    output_types=(tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)
).repeat(epochs)

In [6]:
nblocks = 4
kernel_size = [5, 3, 3, 3, 3]
filters = 16
dilation_rate = [1, 1, 2, 4, 8]

model = denoiser.BinDenoiser(
    nblocks=nblocks,
    kernel_size=kernel_size,
    filters=filters,
    dilation_rate=dilation_rate,
)

optimizer = tf.optimizers.Adam(1e-4)

def kld(y, yhat):
    eps = 1e-14
    out = - y * tf.math.log((yhat + eps) / (y + eps))
    return tf.reduce_mean(tf.reduce_sum(out, -1))

# @tf.function(experimental_relax_shapes=True)
def train_step(logits_x, y, mask):
    with tf.GradientTape() as tape:
        logits_yhat = model(logits_x, training=True, mask=mask)
        yhat = denoiser.masked_softmax(logits_yhat, mask, -1)
        loss = kld(yhat, y) + kld(y, yhat)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss

In [7]:
logits_x, x, logits_y, y, mask = next(iter(dataset))

In [13]:
mask.shape

TensorShape([32, 392])

In [8]:
step = 0
log_every = 100

for batch_data in dataset:
    logits_x, x, logits_y, y, mask = batch_data
    loss = train_step(logits_x, y, mask)
    
    if step % log_every == 0:
        logger.write_metric({"global_step": step, "loss": float(loss)})
    
    step += 1

{"global_step": 0, "loss": 32.48406219482422, "logbook_id": "0", "logbook_timestamp": "07:07:50PM CDT Sep 03, 2020", "logbook_type": "metric"}


InvalidArgumentError: padded_shape[0]=402 is not divisible by block_shape[0]=4 [Op:SpaceToBatchND]